In [36]:
import pandas as pd
import numpy as np

In [37]:
df = pd.read_csv("NASA Exoplanet Data/NASA Exoplanet Archive - Planetary Systems Data.csv")

C:\Users\jarne\AppData\Local\Temp\ipykernel_29820\3066643026.py:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("NASA Exoplanet Data/NASA Exoplanet Archive - Planetary Systems Data.csv")


In [38]:
columns_to_select = ['rowid', 'pl_name', 'st_teff','st_lum','sy_dist','pl_orbsmax','pl_orbeccen','pl_orbper','pl_masse','pl_rade','pl_dens','pl_eqt']
selected_columns = df[columns_to_select]

print(selected_columns)
print(selected_columns.isnull().sum())
print((selected_columns.isnull().sum() / len(df)) * 100)

       rowid    pl_name  st_teff  st_lum   sy_dist  pl_orbsmax  pl_orbeccen  \
0          1   11 Com b      NaN     NaN   93.1846       1.210          NaN   
1          2   11 Com b   4742.0   2.243   93.1846       1.290        0.231   
2          3   11 Com b   4874.0   1.978   93.1846       1.178        0.238   
3          4   11 UMi b   4213.0     NaN  125.3210       1.530        0.080   
4          5   11 UMi b      NaN     NaN  125.3210       1.510          NaN   
...      ...        ...      ...     ...       ...         ...          ...   
36457  36458  ups And d      NaN     NaN   13.4054       2.570        0.269   
36458  36459  ups Leo b   4836.0   1.800   52.5973       1.180        0.320   
36459  36460   xi Aql b      NaN     NaN   56.1858       0.580          NaN   
36460  36461   xi Aql b   4780.0   1.839   56.1858       0.680        0.000   
36461  36462   xi Aql b   4841.0   1.767   56.1858       0.625        0.059   

        pl_orbper  pl_masse  pl_rade  pl_dens  pl_e

In [39]:
INNER_CONSTANT = 1.1
OUTER_CONSTANT = 0.53

def habitable_zone(luminosity):
    """Calculate the habitable zone boundaries in AU."""
    r_inner = np.sqrt(luminosity / INNER_CONSTANT)
    r_outer = np.sqrt(luminosity / OUTER_CONSTANT)
    return r_inner, r_outer

In [40]:
df = selected_columns.dropna(subset=['st_lum'])

hz_inner = []
hz_outer = []

for luminosity in df['st_lum']:
    r_inner, r_outer = habitable_zone(luminosity)
    hz_inner.append(r_inner)
    hz_outer.append(r_outer)

df['hz_inner'] = hz_inner
df['hz_outer'] = hz_outer

C:\Users\jarne\AppData\Local\Temp\ipykernel_29820\3857525657.py:6: RuntimeWarning: invalid value encountered in sqrt
  r_inner = np.sqrt(luminosity / INNER_CONSTANT)
C:\Users\jarne\AppData\Local\Temp\ipykernel_29820\3857525657.py:7: RuntimeWarning: invalid value encountered in sqrt
  r_outer = np.sqrt(luminosity / OUTER_CONSTANT)
C:\Users\jarne\AppData\Local\Temp\ipykernel_29820\689554323.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hz_inner'] = hz_inner
C:\Users\jarne\AppData\Local\Temp\ipykernel_29820\689554323.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [ ]:
df['is_habitable'] = (df['pl_orbsmax'] >= df['hz_inner']) & (df['pl_orbsmax'] <= df['hz_outer'])

habitable_planets = df[df['is_habitable']]

print(habitable_planets)
habitable_planets.to_csv("habitable_planets.csv", index=True)

       rowid       pl_name  st_teff  st_lum    sy_dist  pl_orbsmax  \
23        24      17 Sco b  4157.00   2.254   124.9530      1.4500   
33        34      24 Sex b  5098.00   1.164    72.0691      1.3330   
189      190  BD+20 2457 c  4137.00   3.170  1539.7100      2.0100   
196      197   BD+48 740 b  4534.00   1.640   666.5890      1.7000   
1774    1775    HD 10180 f  5911.00   0.173    38.9607      0.4929   
...      ...           ...      ...     ...        ...         ...   
36398  36399    nu Oct A b  4860.00   1.230    21.1999      1.2500   
36399  36400      nu Oph b  4886.00   2.039    46.2107      1.7900   
36400  36401      nu Oph b  4887.00   1.895    46.2107      1.7650   
36401  36402      nu Oph b  4928.00   2.090    46.2107      1.9000   
36451  36452     ups And c  6105.51   0.525    13.4054      0.8320   

       pl_orbeccen  pl_orbper    pl_masse  pl_rade  pl_dens  pl_eqt  hz_inner  \
23          0.0600     578.38         NaN      NaN      NaN     NaN  1.431465 

C:\Users\jarne\AppData\Local\Temp\ipykernel_29820\91623473.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_habitable'] = (df['pl_orbsmax'] >= df['hz_inner']) & (df['pl_orbsmax'] <= df['hz_outer'])
